In [ ]:
!pip install tenseal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 28.4 MB/s eta 0:00:00


In [ ]:
import tenseal as ts

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
context.generate_galois_keys()
context.global_scale=2**40

In [ ]:
import numpy as np
import time
import psutil
import matplotlib.pyplot as plt
from scipy.special import expit
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from threading import Thread, Lock

In [ ]:
X_train = pd.read_csv('X_train2.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train2.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test2.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test2.csv')
y_test=y_test.to_numpy().reshape(-1)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Lock for controlling CPU monitoring
monitor_lock = Lock()

In [ ]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [ ]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [ ]:
# def initialize_parameters(n_features):
#     w = np.zeros(n_features)
#     b = 0
#     return w, b
weights_enc= np.zeros(X_train.shape[1])
bias_enc=0
lr=0.01
C=1
C2=ts.ckks_vector(context,[C])
n_samples, n_features = X_train.shape
weights_enc=ts.ckks_vector(context,weights_enc)
n_samples, n_features

(1080, 22)

In [ ]:
weights_enc

In [ ]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [ ]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [ ]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        # with monitor_lock:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [ ]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 1400

In [ ]:
time.sleep(15)

In [ ]:
def marginal_decrypt(margin):
  margin_x=None
  with monitor_lock:
    margin_x=margin.decrypt()[0]
  if margin_x>0:
    return True
  else:
    return False

In [ ]:
results={}
def train_svm(enc_X_train,enc_y_train,weights,bias,lr,C):
  train_time_start = time.time()
  weights_enc=weights
  bias_enc=bias

  for epoch in range(3):
    for i in range(n_samples):
      xi=enc_X_train[i]
      yi=enc_y_train[i]

      rog=xi.dot(weights_enc)
      rog=relinearlizing_scalar(rog)
      margins = 1 - (yi * (rog+bias_enc))
      if marginal_decrypt(margins) > 0:
        w=C*yi
        w=relinearlizing_scalar(w)
        y=w*xi
        # y=relinearlizing_vector(y)
        x=weights_enc-y
        x=relinearlizing_vector(x)

        weights_enc=weights_enc-lr*x
        bias_enc=bias_enc-lr*(-C*yi)
        weights_enc=relinearlizing_vector(weights_enc)
        bias_enc=relinearlizing_scalar(bias_enc)
      else:
        weights_enc=weights_enc-lr*weights_enc
        weights_enc=relinearlizing_vector(weights_enc)

      print(f'epoch {epoch+1}, sample {i} executed')

  results['weights']=weights_enc
  results['bias']=bias_enc
  results['train_time']=time.time()-train_time_start
  print(results['train_time'])

In [ ]:
train_thread = Thread(target=train_svm, args=(enc_X_train, enc_y_train, weights_enc, bias_enc, lr, C))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

CPU utilization: 96.5% | RAM utilization: 12.239011886126972%
epoch 1, sample 0 executed
epoch 1, sample 1 executed
CPU utilization: 93.5% | RAM utilization: 12.261012642760381%
epoch 1, sample 2 executed
epoch 1, sample 3 executed
CPU utilization: 98.4% | RAM utilization: 12.267001904826111%
epoch 1, sample 4 executed
epoch 1, sample 5 executed
epoch 1, sample 6 executed
epoch 1, sample 7 executedCPU utilization: 100.6% | RAM utilization: 12.270974681673732%

epoch 1, sample 8 executed
epoch 1, sample 9 executed
epoch 1, sample 10 executed
CPU utilization: 80.7% | RAM utilization: 12.270974681673732%
epoch 1, sample 11 executed
epoch 1, sample 12 executed
epoch 1, sample 13 executed
CPU utilization: 100.6% | RAM utilization: 12.270974681673732%
epoch 1, sample 14 executed
epoch 1, sample 15 executed
epoch 1, sample 16 executed
CPU utilization: 100.1% | RAM utilization: 12.27494745852135%
epoch 1, sample 17 executed
epoch 1, sample 18 executed
epoch 1, sample 19 executed
CPU utilizatio

In [ ]:
weights_unenc=results['weights'].decrypt()
print(weights_unenc)
bias_unenc=results['bias'].decrypt()
print(bias_unenc)

[0.08552170005577103, 0.09155179874042949, -0.07277440036688806, 0.14697610006150288, 0.009888600449886972, 0.032819201746604944, -0.13227209994324662, 0.01708579999141824, 0.29775719961973823, 0.02128970128212941, 0.10422929971590207, 0.17704119965187173, 0.10354850115024335, 0.15673349777714268, 0.08239530010347601, 0.10415200060931204, 0.06068169878181582, -0.07345239898566364, -0.0706329021855885, -0.1874954997354903, -0.1862802989651235, -0.03404429917936096]
[0.18000000021056642]


In [ ]:
enc_X_test=[]
for i in X_test:
  p=ts.ckks_vector(context,i)
  enc_X_test.append(p)

In [ ]:
def predict(X_test,weights_enc,bias_enc):
  y_pred=[]
  for xi in X_test:
    rog=xi.dot(weights_enc)+bias_enc
    hell=marginal_decrypt(rog)
    if hell>0:
      y_pred.append(1)
    else:
      y_pred.append(-1)
  return y_pred

In [ ]:
test_time_start=time.time()
y_pred=predict(enc_X_test,results['weights'],results['bias'])
test_time=time.time()-test_time_start
results['test_time']=test_time
print(results['test_time'])

8.606898784637451


In [ ]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.8

In [ ]:
import joblib
joblib.dump((results['weights'].decrypt(), results['bias'].decrypt()[0], accuracy, y_pred, cpu_seconds, cpu_usages, ram_usages, results['train_time'], results['test_time']), 'variables_ckks.pkl')

['variables_ckks.pkl']